In [1]:
#import libraries
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import geopy.distance
import re
import os

url for rides data: http://opendata.dc.gov/datasets?q=taxicabs

zip code shape files for all three states: https://www.census.gov/geo/maps-data/data/cbf/cbf_zcta.html

In [9]:
#move the files from the cwd to the 
file_list =['taxitrip_2015.zip','taxitrip_2016.zip','taxitrip_2017.zip', 'merged_CTs.zip']
            
for file in file_list:
    os.system('mv '+file+' '+os.getcwd()+'/Data')

In [12]:
for file in file_list:
    os.system('unzip '+os.getcwd()+'/Data/'+file)

In [15]:
os.getcwd()

'/nfshome/cb4102/MOB'

In [73]:
zipcodes = gpd.read_file('Data/cb_2017_us_zcta510_500k.shp')
zipcodes.head()

,ZCTA5CE10,AFFGEOID10,GEOID10,ALAND10,AWATER10,geometry
0,35442,8600000US35442,35442,610213891,10838694,"(POLYGON ((-88.252618 32.92675, -88.249724 32...."
1,85365,8600000US85365,85365,3545016067,9766486,"(POLYGON ((-114.684663 32.687389, -114.676063 ..."
2,71973,8600000US71973,71973,204670474,1264709,"POLYGON ((-94.46643176650841 34.330735, -94.46..."
3,95445,8600000US95445,95445,221559097,7363179,"POLYGON ((-123.643051 38.851233, -123.641401 3..."
4,06870,8600000US06870,06870,5945321,3841130,"POLYGON ((-73.58765810889219 41.0076522198727,..."


In [74]:
zipcodes.shape

(33144, 6)

In [30]:
pd.set_option('display.max_columns', 500)

In [98]:
#load each file as a dataframe, filter it and geotag it 
path = 'Data/'

#list of files to load individuallly
file_list = ['Apr 2015 Trips.rpt','May 2015 Trips.rpt','Jun 2015 Trips.rpt','Jul 2015 Trips.rpt','Aug 2015 Trips.rpt',
            'Sep 2015 Trips.rpt','Oct 2015 Trips.rpt','Nov 2015 Trips.rpt','Dec 2015 Trips.rpt','Jan 2016 Trips.rpt',
            'Feb 2016 Trips.rpt','Mar 2016 Trips.rpt','Apr 2016 Trips.rpt','May 2016 Trips.rpt','Jun 2016 Trips.rpt',
            'Jul 2016 Trips.rpt','Aug 2016 Trips.rpt','Sep 2016 Trips.rpt','Nov 2016 Trips.rpt','Dec 2016 Trips.rpt',
            'Jan 2017 Trips.rpt','Feb 2017 Trips.rpt','Mar 2017 Trips.rpt','Apr 2017 Trips.rpt','May 2017 Trips.rpt',
            'Jun 2017 Trips.rpt','Jul 2017 Trips.rpt','Aug 2017 Trips.rpt','Sep 2017 Trips.rpt','Oct 2017 Trips.rpt',
            'Nov 2017 Trips.rpt','Dec 2017 Trips.rpt']

#list of columns to keep from the raw data
col_list = ['StartDateTime.1','EndDateTime','OriginStreetName','OriginLatitude','OriginLongitude',
                        'DestinationStreetName','DestinationLatitude','DestinationLongitude']

df_list = [] #initialize list to store dataframes

for i in file_list:
    df = pd.read_csv(path+i,sep = '|',error_bad_lines=False) #load month file
    df = df[col_list] #filter to column list
    df['id'] = df.index + 1 #create id column
    
    #clean dataframe for converting to gpd
    geo_test = df[['id','OriginLatitude','OriginLongitude','DestinationLatitude','DestinationLongitude']]
    #clean geo data - drop NaNs and 
    geo_test.dropna(how='any',inplace=True)
    geo_test = geo_test[(geo_test.T != 0).any()]
    
    #convert to geopandas df and perform spatial join to get pickup zip
    geometry_pickup = [Point(xy) for xy in zip(geo_test['OriginLongitude'], geo_test['OriginLatitude'])]
    crs = {'init': 'epsg:4269'}
    pickup_geo = gpd.GeoDataFrame(geo_test, crs=crs, geometry=geometry_pickup)
    
    geo_pickup = gpd.sjoin(pickup_geo, zipcodes, how='left', op='intersects')
    geo_pickup = pd.DataFrame(geo_pickup[['id','ZCTA5CE10']])
    geo_pickup.rename(columns={'ZCTA5CE10':'pickup_zip'},inplace=True)
    
    #Get drop-off zip from zip code spatial join
    geometry_dropoff = [Point(xy) for xy in zip(geo_test['DestinationLongitude'], geo_test['DestinationLatitude'])]
    crs = {'init': 'epsg:4269'}
    dropoff_geo = gpd.GeoDataFrame(geo_test, crs=crs, geometry=geometry_dropoff)
    
    geo_dropoff = gpd.sjoin(dropoff_geo, zipcodes, how='left', op='intersects')
    geo_dropoff = pd.DataFrame(geo_dropoff[['id','ZCTA5CE10']])
    geo_dropoff.rename(columns={'ZCTA5CE10':'dropoff_zip'},inplace=True)
    
    geo = pd.merge(left=geo_pickup,right=geo_dropoff,left_on='id',right_on='id')
    df = pd.merge(left=df,right=geo,how='left',left_on='id',right_on='id')
    
    df_list.append(df)
    
    print(i+" is loaded")
    
dc_rides = pd.concat(df_list)

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Apr 2015 Trips.rpt is loaded
May 2015 Trips.rpt is loaded
Jul 2015 Trips.rpt is loaded
Aug 2015 Trips.rpt is loaded
Sep 2015 Trips.rpt is loaded
Oct 2015 Trips.rpt is loaded
Nov 2015 Trips.rpt is loaded
Dec 2015 Trips.rpt is loaded
Jan 2016 Trips.rpt is loaded
Feb 2016 Trips.rpt is loaded
Mar 2016 Trips.rpt is loaded
Apr 2016 Trips.rpt is loaded
May 2016 Trips.rpt is loaded
Jun 2016 Trips.rpt is loaded
Jul 2016 Trips.rpt is loaded
Aug 2016 Trips.rpt is loaded
Sep 2016 Trips.rpt is loaded
Nov 2016 Trips.rpt is loaded
Dec 2016 Trips.rpt is loaded


/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,5,15,19,22,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Jan 2017 Trips.rpt is loaded


b'Skipping line 521111: expected 31 fields, saw 32\nSkipping line 521221: expected 31 fields, saw 32\n'
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,5,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


...

Oct 2017 Trips.rpt is loaded


b'Skipping line 83789: expected 31 fields, saw 32\n'
b'Skipping line 169475: expected 31 fields, saw 32\nSkipping line 193945: expected 31 fields, saw 32\n'
b'Skipping line 274767: expected 31 fields, saw 32\n'
b'Skipping line 321077: expected 31 fields, saw 32\n'
b'Skipping line 350916: expected 31 fields, saw 32\n'
b'Skipping line 463204: expected 31 fields, saw 32\n'
b'Skipping line 556772: expected 31 fields, saw 32\n'
b'Skipping line 836699: expected 31 fields, saw 32\n'


Nov 2017 Trips.rpt is loaded


b'Skipping line 866: expected 31 fields, saw 32\nSkipping line 1011: expected 31 fields, saw 32\n'
b'Skipping line 65592: expected 31 fields, saw 32\n'
b'Skipping line 106959: expected 31 fields, saw 32\n'
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,15,19,22,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Dec 2017 Trips.rpt is loaded


In [99]:
dc_rides.head()
print(dc_rides.shape)

(36488272, 11)


In [100]:
dc_rides.head()

,StartDateTime.1,EndDateTime,OriginStreetName,OriginLatitude,OriginLongitude,DestinationStreetName,DestinationLatitude,DestinationLongitude,id,pickup_zip_x,pickup_zip_y
0,2015-04-24 11:09:49.5900000 +00:00,2015-04-24 11:20:39.2066667 +00:00,1120 Nw L St Washington Dc 20037,38.90364,-77.02718,810 Nw 1st St Washington Dc 20001,38.90036,-77.01225,1,20005,20001
1,2015-04-24 12:10:31.9666667 +00:00,2015-04-24 12:23:08.1700000 +00:00,730 Nw 12th St Washington Dc 20005,38.89867,-77.02763,2006 Nw P St Washington Dc 20036,38.90966,-77.04531,2,20001,20036
2,2015-04-24 13:47:14.6466667 +00:00,2015-04-24 14:06:45.7066667 +00:00,740 Nw 12th St Washington Dc 20005,38.89892,-77.02846,Arland D Williams Memorial Brg @ S George Wash...,38.87328,-77.04203,3,20005,NaN
3,2015-04-24 12:41:21.0266667 +00:00,2015-04-24 12:57:53.6166667 +00:00,1530 Nw K St Washington Dc 20001,38.90294,-77.03515,290 Sw Raoul Wallenberg Pl Washington Dc 20024,38.88566,-77.03374,4,20005,20024
4,2015-04-16 16:06:54.9766667 +00:00,2015-04-16 16:13:48.4333333 +00:00,180 Nw 22nd St Washington Dc 20037,38.89319,-77.04884,1030 Nw New Hampshire Ave Washington Dc 20036,38.90317,-77.04966,5,20418,20037


In [103]:
#drop unneeded columns
dc_rides.drop(['OriginLatitude','OriginLongitude','DestinationLatitude','DestinationLongitude','id'],axis=1,inplace=True)
dc_rides.to_csv("all_rides_geotagged.csv")
dc_rides.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36488272 entries, 0 to 730696
Data columns (total 6 columns):
StartDateTime.1          object
EndDateTime              object
OriginStreetName         object
DestinationStreetName    object
pickup_zip_x             object
pickup_zip_y             object
dtypes: object(6)
memory usage: 1.9+ GB


In [105]:
dc_rides['dest_zip']= dc_rides['DestinationStreetName'].str[-5:]
dc_rides['org_zip']= dc_rides['OriginStreetName'].str[-5:]

In [108]:
dc_rides['StartDateTime.1'] = pd.to_datetime(dc_rides['StartDateTime.1'],errors='coerce')
dc_rides['EndDateTime'] = pd.to_datetime(dc_rides['EndDateTime'],errors='coerce')

In [109]:
dc_rides['YEAR'] = pd.DatetimeIndex(dc_rides['StartDateTime.1']).year #create a year column
dc_rides['DOY'] = pd.DatetimeIndex(dc_rides['StartDateTime.1']).dayofyear #create a day of year column
dc_rides['HOUR']= pd.DatetimeIndex(dc_rides['StartDateTime.1']).hour #create an hour of day column
dc_rides['DAY']= pd.DatetimeIndex(dc_rides['StartDateTime.1']).day

In [112]:
dc_rides.to_csv('pre_agg.csv')

In [122]:
#clean up data set
dc_rides.drop(['dest_zip','org_zip','pickup_match','StartDateTime.1','EndDateTime','OriginStreetName',
               'DestinationStreetName'],axis=1,inplace=True)
dc_rides.rename({'pickup_zip_x':'PICKUPZIP','pickup_zip_y':'DROPOFFZIP'},inplace=True)
dc_rides.dropna(how='any',inplace=True)

In [133]:
dc_rides = dc_rides.groupby(['PICKUPZIP','DROPOFFZIP','YEAR','DOY','HOUR'])['DAY'].count()

In [135]:
dc_rides = pd.DataFrame(dc_rides)

In [137]:
dc_rides.reset_index(inplace=True)

In [147]:
dc_rides.to_csv('dc_rides_2015_2017.csv')